In [26]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json 

In [27]:
dfWS=pd.read_excel("water security.xlsx",skiprows=[0])

In [28]:
dfWS.head(4)
# testcase
#
dfWS=dfWS[:10]

In [29]:
len(dfWS)

10

In [30]:
dfWS.keys()

Index(['Rank,Publication ID,DOI,PMID,PMCID,Title,Source title,Anthology title,Publisher,MeSH terms,Publication Date,PubYear,Volume,Issue,Pagination,Open Access,Publication Type,Authors,Corresponding Author,Authors Affiliations,Research Organizations - standardized,GRID IDs,City of Research organization,State of Research organization,Country of Research organization,Funder,Funder Group,Funder Country,UIDs of supporting grants,Supporting Grants,Times cited,Recent citations,RCR,FCR,Altmetric,Source Linkout,Dimensions URL,FOR (ANZSRC) Categories,RCDC Categories,HRCS HC Categories,HRCS RAC Categories,ICRP Cancer Types,ICRP CSO Categories,Units of Assessment,Sustainable Development Goals',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 519', 'Unnamed: 520', 'Unnamed: 521', 'Unnamed: 522',
       'Unnamed: 523', 'Unnamed: 524', 'Unnamed: 525', 'Unnamed: 526',
       'Unname

# Webseiten auswerten

In [31]:
pubkeys=[
    'title',
    'aff_org_name',
#    'affiliations_json',
#    'author_list',
     'researcher_dim_id',
     'researcher_dim_count',
     'journal_title',
     'language',
     'abstract',
     'open_access',
     'publisher',
#     'pub_class',
#    'aff_city_geonames_ids',
     'aff_city_name',
     'aff_country_name',
#     'aff_org_id',
     'doi',
#     'id',
     'pub_date',
     'pub_year',
     'times_cited',
     'altmetric_id',
     'altmetric',
#     'score',
#     'affiliations_details',
     'authors_full',
 #    'journal'
        ]

In [32]:
## get html pages from dimension
def gethtml(x):
    html=""
    url=x["Dimensions URL"]
    if url!=None:
        html = requests.get(url).content
    return(html)
## create a soup column
def getsoup(x):
    html=x["htmldimension"]
    return(BeautifulSoup(html,"html"))
## create a dict column
def getdimdict(x):
    s=x["soup"]
    datadoc=s.find("div")
    d=""
    if datadoc==None:
        attr=""
    else:
        attr=datadoc.get("data-doc")
        if attr:
            d=json.loads(attr)
    return(d)
def makedic(x):
    dicpub=dict()
    for i in pubkeys:
        v=x.get(i)
        dicpub[i]=v
    return(dicpub)

In [33]:
dfWS["htmldimension"]=dfWS.apply(lambda x: gethtml(x),axis=1)

KeyError: ('Dimensions URL', 'occurred at index 0')

In [18]:
dfWS["soup"]=dfWS.apply(lambda x:getsoup(x),axis=1)

KeyError: ('htmldimension', 'occurred at index 0')

In [19]:
dfWS["dimdict"]=dfWS.apply(lambda x:getdimdict(x),axis=1)

KeyError: ('soup', 'occurred at index 0')

In [20]:
dfWS["seldict"]=dfWS["dimdict"].apply(makedic)

KeyError: 'dimdict'

In [21]:
lidi=[]
def newpub(x):
    di=dict()
    d=x["seldict"]
    di["DOI"]=x["DOI"]
    di["title"]=x["Title"]
    di["authors"]=d["authors_full"]
    di["publisher"]=d["publisher"]
    di["source"]=x["Source title"]
    di["aff_org_name"]=d["aff_org_name"]
    di["aff_country"]=d["aff_country_name"]
    di["pub_date"]=d["pub_date"]
#    di["month"]=x["month"]
#    di["day"]=x["day"]
    di["abstract"]=d["abstract"]
    di["openaccess"]=d["open_access"]
    di["di_URL"]=x["Dimensions URL"]
    lidi.append(di)

In [22]:
dfWSd.apply(lambda x:newpub(x),axis=1)
dfRemLit=pd.DataFrame(lidi)
dfRemLit.head(3)

NameError: name 'dfWSd' is not defined

In [23]:
dfRemLit.to_json("dfRemLit.json")

NameError: name 'dfRemLit' is not defined